## About
use this script to make a GIF that shows canal flows using NICFI (5m) satellite imagery.<br>
pre-requisites:
- google earth engine access
- nicfi imagery access (alternatively use just publicly available sentinel 2 imagery)

## Imports

In [60]:
import geemap
import ee

# Initialize the Earth Engine module.
ee.Initialize()

import datetime
from dateutil.relativedelta import relativedelta
from datetime import timedelta as delta

## Feature & Image Collections

In [6]:
# Image Collections
nicfiColl = ee.ImageCollection("projects/planet-nicfi/assets/basemaps/asia")
s2Coll = ee.ImageCollection("COPERNICUS/S2_SR")
s1Coll = ee.ImageCollection("COPERNICUS/S1_GRD")
dwColl = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")

# Feature Collections & Geometries
nrbc = ee.FeatureCollection("projects/ee-craigdsouza/assets/canals/nrbc_d9_d18")
nrbcBBox = nrbc.geometry().bounds().buffer(2000)
nrbcBuffer = nrbc.geometry().buffer(20)
geometry = ee.Geometry.Polygon(
        [[[76.35728463743025, 16.21064824268761],
          [76.35728463743025, 16.198944508521084],
          [76.38449296567732, 16.198944508521084],
          [76.38449296567732, 16.21064824268761]]], None, False);


## Visualization Params

In [7]:
visParamsNICFI = {'bands':["R","G","B"],'min':400,'max':2000}
visParamsS2 = {'bands':["R","G","B"],'min':500,'max':3500}

ndwiPalette = ['fff','90bdf3','0600e8']
visParamsW = {'bands':['NDWI'],'palette':ndwiPalette,'min':-0.5,'max':0}

ndviPalette = ['f00','ff0','0f0']    #['EB2121','FBFF59','3CAE12']
visParamsV = {'bands':['NDVI'],'palette':ndviPalette,'min':0,'max':0.6}

dwPalette = ['419bdf', '397d49', '88b053', '7a87c6', 'e49635', 'dfc35a', 'c4281b','a59b8f', 'b39fe1']
dwVis = {'bands':['label'],'palette':dwPalette,'min':0,'max':8}

## Functions

In [8]:
def prepND(image):
  ndwi = image.normalizedDifference(["G","N"])
  ndvi = image.normalizedDifference(["N","R"])
  aoi = nrbcBuffer
  ndwi = ndwi.clip(aoi).rename('NDWI') #.updateMask(mask)
  ndvi = ndvi.clip(aoi).rename('NDVI') #.updateMask(mask)
  return image.addBands([ndwi,ndvi]) #.updateMask(mask)

In [42]:
# Make NICFI Canals Subset for one hydrological year
def makeNICFISubset(year):
  start_date = datetime.date(year, 6, 1)
  end_date = datetime.date(year+1, 5, 1)

  dates = []
  current_date = start_date
  while current_date <= end_date:
    next_date = current_date + relativedelta(months=1)  #.strftime('%Y-%m-%d')
    dates.append((current_date, next_date))
    current_date += relativedelta(months=1)

  ndColl = []
  for i,(start,end) in enumerate(dates):
    start = start.strftime('%Y-%m-%d') # e.g. '2022-06-01'
    end = end.strftime('%Y-%m-%d') # e.g. '2022-07-01'
    monthlyMosaic = nicfiColl.filterDate(start,end).first()
    ndMosaic = prepND(monthlyMosaic)
    ndColl.append(ndMosaic)
  return ee.ImageCollection(ndColl)

In [118]:
# Make Sentinel-2 Canals Subset for one hydrological year
def makeS2Subset(year):
    start_date = datetime.date(year, 6, 1)  #.strftime('%Y-%m-%d')
    end_date = datetime.date(year+1, 6, 1)  #.strftime('%Y-%m-%d')
    # list of dates from start_date to end_date
    dates = []
    current_date = start_date
    while current_date <= end_date:
      dates.append(current_date)
      current_date += delta(days=1)
    # print(len(dates)) #366

    s2y = []
    
    for i in range(len(dates)-1):
      start = dates[i].strftime('%Y-%m-%d')
      end = dates[i+1].strftime('%Y-%m-%d')
      subset = s2Coll.filterDate(start,end).filterBounds(geometry).select(['B4','B3','B2','B8'],['R','G','B','N'])
      
      if subset.size().getInfo() > 0:
        #  print(f"{subset.size().getInfo()} images available for {start}")
         dt = subset.first().get('system:time_start')
         s2img = subset.median().set('system:time_start',dt)
         s2y.append(s2img)
    
    s2y = ee.ImageCollection(s2y)
    # print(s2y.size().getInfo()) # no more than 52
    s2y = s2y.map(prepND)
    return s2y

In [114]:
# Blend NDWI canal images with basemap  
def blendImages(image,basemap):
  # Add visualization parameters to your image
  ndwiPalette = ['fff','90bdf3','0600e8']
  visParamsW = {'bands':['NDWI'],'palette':ndwiPalette,'forceRgbOutput':True,'min':-0.5,'max':0}
  visualizedImage = image.visualize(**visParamsW).set('system:time_start',image.get('system:time_start'))

  # Blend your image with the basemap
  return basemap.blend(visualizedImage).set('system:time_start',image.get('system:time_start'))

## Prepare Images

In [12]:
# Dynamic World
dw = dwColl.filterDate('2022-06-01','2023-06-01').filterBounds(nrbcBBox).mode().select('label').clip(nrbcBBox)

In [105]:
# NICFI Canals
nicfi_canals_2022 = makeNICFISubset(2022)

In [14]:
# Sentinel 2 Basemap
s2 = s2Coll.filterDate('2022-06-01','2023-06-01').filterBounds(nrbcBBox).median().select(['B4','B3','B2','B8'],['R','G','B','NIR']).clip(nrbcBBox)

In [119]:
# Sentinel 2 Canals
s2_canals_2022 = makeS2Subset(2022)
print(s2_canals_2022.size().getInfo())  # takes a long time to run, for 1year approx 3.5min

73


## View Layers in map

In [128]:
# Create a map
Map = geemap.Map()

# Add the Dynamic World image to the map
# Map.addLayer(dw, dwVis, 'Dynamic World')

# Add the Sentinel 2 image to the map
Map.addLayer(s2_canals_2022.first(), visParamsS2, 'Sentinel 2')

# Add the first NICFI image to the map
# Map.addLayer(nicfi_canals_2022.first(), visParamsW, 'NICFI Image')

# Center the map at the displayed layers
Map.centerObject(nrbcBBox, 12)

# Display the map
Map

Map(center=[16.356900062746625, 76.78932755308384], controls=(WidgetControl(options=['position', 'transparent_…

## Prepare GIF

In [125]:
scale = nicfi_canals_2022.first().toFloat().projection().nominalScale()
projection = nicfi_canals_2022.first().toFloat().projection()

scaleS2 = s2_canals_2022.first().toFloat().projection().nominalScale()
projectionS2 = s2_canals_2022.first().toFloat().projection()

In [120]:
# Load a basemap (e.g., a Landsat image)
basemap = s2.select(['R', 'G', 'B']).visualize(bands=['R','G','B'],min=1000,max=4000) # Adjust visualization parameters as needed

# Labeling your collection of NDWI images
labeledImages = nicfi_canals_2022.map(lambda image: blendImages(image, basemap))
labeledImagesS2 = s2_canals_2022.map(lambda image: blendImages(image, basemap))

In [122]:
# Get the list of dates
# dates = labeledImages.filterBounds(geometry).aggregate_array('system:time_start').getInfo()
dates_s2 = labeledImagesS2.filterBounds(geometry).aggregate_array('system:time_start').getInfo()

# Convert the dates to the desired format
# formatted_dates = [datetime.datetime.fromtimestamp(date/1000).strftime('%B-%Y') for date in dates]
formatted_dates = ["June-2022","July-2022","August-2022","September-2022","October-2022","November-2022",
                   "December-2022","January-2023","February-2023","March-2023","April-2023","May-2023"]
formatted_dates_s2 = [datetime.datetime.fromtimestamp(date/1000).strftime('%Y-%m-%d') for date in dates_s2]

# Print the formatted dates
# print(formatted_dates)
print(formatted_dates_s2)

['2022-06-05', '2022-06-10', '2022-06-15', '2022-06-20', '2022-06-25', '2022-06-30', '2022-07-05', '2022-07-10', '2022-07-15', '2022-07-20', '2022-07-25', '2022-07-30', '2022-08-04', '2022-08-09', '2022-08-14', '2022-08-19', '2022-08-24', '2022-08-29', '2022-09-03', '2022-09-08', '2022-09-13', '2022-09-18', '2022-09-23', '2022-09-28', '2022-10-03', '2022-10-08', '2022-10-13', '2022-10-18', '2022-10-23', '2022-10-28', '2022-11-02', '2022-11-07', '2022-11-12', '2022-11-17', '2022-11-22', '2022-11-27', '2022-12-02', '2022-12-07', '2022-12-12', '2022-12-17', '2022-12-22', '2022-12-27', '2023-01-01', '2023-01-06', '2023-01-11', '2023-01-16', '2023-01-21', '2023-01-26', '2023-01-31', '2023-02-05', '2023-02-10', '2023-02-15', '2023-02-20', '2023-02-25', '2023-03-02', '2023-03-07', '2023-03-12', '2023-03-17', '2023-03-22', '2023-03-27', '2023-04-01', '2023-04-06', '2023-04-11', '2023-04-16', '2023-04-21', '2023-04-26', '2023-05-01', '2023-05-06', '2023-05-11', '2023-05-16', '2023-05-21', '2023

In [129]:
print(scaleS2.getInfo(),projectionS2.getInfo())
print(scale.getInfo(),projection.getInfo())

111319.49079327357 {'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}
4.777314267160193 {'type': 'Projection', 'crs': 'EPSG:3857', 'transform': [4.777314267160193, 0, -20037508.340000007, 0, -4.777314267160193, 3522218.2628982645]}


In [130]:
# Set the export parameters
export_params = {
    'region': geometry,
    'scale': scale, # 10m/4.777314267160193 m
    'crs': projection, # EPSG:3857
    'folder': 'EEImageExports',
    'description': 'S2_Flows',
    'framesPerSecond': 5
}

# Export the labeledImages as a video
# ee.batch.Export.video.toDrive(labeledImages, **export_params).start()
ee.batch.Export.video.toDrive(labeledImagesS2, **export_params).start()

## Add labels to video

In [3]:
import cv2
from moviepy.editor import ImageSequenceClip
import os

In [103]:
# Function to add month and year to the frames
def label_frames(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError("Error opening video file")

    frames = []
    
    for frame_num in range(12):  # Assuming 12 frames, one for each month
        ret, frame = cap.read()
        if not ret:
            break

        # Filter month and year
        label = formatted_dates[frame_num]

        # Add label to the frame
        cv2.putText(frame, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        frames.append(frame)

    cap.release()

    # Compile frames into a video
    clip = ImageSequenceClip([cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in frames], fps=1)
    clip.write_videofile(output_path, codec='libx264')

In [ ]:
# Usage
video_path = "C:\\path\\to\\your\\video.mp4"
output_path = "C:\\path\\to\\your\\videoLabeled.mp4"
label_frames(video_path, output_path)